## Setup

In [1]:
!pip install -qq timm
!pip install -qq fastai
!pip install -qq pynvml

import timm 
from fastai.vision.all import *

## Data loading

In [ ]:
# Load data from image folder
dls = ImageDataLoaders.from_folder(path, 
        valid_pct=0.2,
        item_tfms=Resize(480),
        batch_tfms=aug_transforms(size=224, min_scale=0.75),
        bs=64)
dls.show_batch(nrows=1, ncols=4)

In [ ]:
# Custom data splitter
def custom_splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=custom_splitter,
                   get_x=get_x, 
                   get_y=get_y)

dsets = dblock.datasets(df)
dls = dblock.dataloaders(df)

## Data exploration

In [ ]:
# check image sizes fast
from fastcore.parallel import *

def get_image_size(o): return PILImage.create(o).size

# files = get_image_files(path)
sizes = parallel(get_image_size, files, n_workers=8)
pd.Series(sizes).value_counts()

## Training

In [ ]:
# learning rate finder
learn.lr_find(suggest_funcs=(minimum, steep, valley))

## Interpretation

In [ ]:
# confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=5)

## Evaluation (test time)

In [ ]:
# evaluate model on whole test dataset
test_dl = learn.dls.test_dl(test_files)
preds, _, decoded = learn.get_preds(dl=test_dl, with_decoded=True)